# "The Average Sydney Suburb - Data munging in Pandas"


- toc: false
- branch: master
- badges: true
- comments: true
- categories: [jupyter]



## The Average Sydney Suburb - Data cleaning

I have recently finished an eDx [course](https://courses.edx.org/courses/course-v1:BerkeleyX+Data8.2x+1T2019a/course/) on inferential statistics from Duke university. This course does a good job of using resampling for inferential statistics.

Applying what I have learned in this course, I want to figure out the most average Sydeny suburbs in terms of the country of birth of person (__CBOP__) and age.

There is a lot of data available via [ABS](https://www.abs.gov.au). I used the [Census table builder](https://www.abs.gov.au/websitedbs/censushome.nsf/home/tablebuilder) to get the CBOP and age data.

In this notebook, I will clean and setup the data for my analysis

In [1]:
#Importing the libraries that I will use
import pandas as pd
import numpy as np
%matplotlib inline

from IPython.display import display

I have downloaded the following files from the 2016 Census that I will need to use for my analysis:
-  **Population**
    -  Total Sydney population
    -  Top 6 CBOP population for Sydney
    -  Total population for all of Sydney suburbs
    -  Top 6 CBOP population for all of Sydney suburbs
-  **Age**
    -  Sydney's age in single digits
    -  Sydney's suburb's age in single digits

As useful as the [Census table builder](https://www.abs.gov.au/websitedbs/censushome.nsf/home/tablebuilder) is, I could not get the data in the format that I needed - hence the need for this cleaning exercise. In the past I have made use of excel and pandas for cleaning data, however in this notebook I will attempt to clean all the data in pandas.

Starting with Sydney's CBOP mix. My aim is to create a table with the top 6 CBOP and Others.

In [2]:
#Read Sydney pop and top 6 cbop
sydney_pop = pd.read_csv('greater_sydney_pop.csv', skiprows = 11)
sydney_cbop = pd.read_csv('top_6_cobp.csv', skiprows = 11)

For the Sydney population table, all I need is the total population value. I can get this from Index 0 and column 4823993

In [3]:
syd_pop = int(sydney_pop.at[0,'4823993'])
print("Sydney's population: ", syd_pop)

Sydney's population:  4823993


Now lets take a look at the Sydney CBOP table

In [4]:
#Using threshold values in drop na to only drop rows with 2 Na values.
#This has the useful side effect of dropping the unnamed column as well
sydney_cbop.dropna(axis = 1, thresh=2, inplace = True)
sydney_cbop.dropna(axis = 0, inplace = True)
#rename BPLP - 4 Digit Level column to Population
sydney_cbop.rename(columns = {sydney_cbop.columns[0]:'Population'}, inplace = True)
#Drop info index
sydney_cbop.drop('INFO', inplace = True)
#convert population numbers to integer
sydney_cbop['Population'] = sydney_cbop['Population'].apply(pd.to_numeric)
#Rename the total index value to CBOP Total. This will be used later to calculate the value of other countries
sydney_cbop.rename({'Total': 'CBOP Total'}, inplace = True)
sydney_cbop.info()
print(sydney_cbop)


<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, England to CBOP Total
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Population  7 non-null      int64
dtypes: int64(1)
memory usage: 112.0+ bytes
                                  Population
England                               151617
Australia                            2752123
China (excludes SARs and Taiwan)      224682
India                                 130579
Vietnam                                81041
New Zealand                            86522
CBOP Total                           3426561


Next step is to add the population of people who are not from any of the 6 top countries

In [5]:
sydney_cbop.loc['CBOP Total'] =  syd_pop - sydney_cbop.at['CBOP Total', 'Population']
sydney_cbop.rename({'CBOP Total' : 'Others'}, inplace = True)
sydney_cbop.Population.sum()

4823996

In [6]:
print(sydney_cbop)
print('\nTotal Sydney Population ', sydney_cbop.Population.sum())

                                  Population
England                               151617
Australia                            2752123
China (excludes SARs and Taiwan)      224682
India                                 130579
Vietnam                                81041
New Zealand                            86522
Others                               1397432

Total Sydney Population  4823996


All set!, we now have a table with the top 6 natonalities (CBOP) of Sydney and an others column. Next step is to repeat this process for every suburb in the greater Sydney area.

_Note: It took me about 2 hours to figure out the best way to clean these two small dataframes and boil it down to a few lines of code. I am putting this here as a reminder to not get overwhelmed when I see posts where things like these seem to be the simplest of steps. Having gone through this exercise, its clear that data cleaning is very important and not as easy as it seems. Atleast when you are just getting started. Keep at it!!_

In [7]:
#Load the top 6 CBOP by Suburb and total population by suburb files
suburb_cbop = pd.read_csv('suburb_top_6_cobp.csv', skiprows = 11)
suburb_pop = pd.read_csv('total_suburb_population.csv', skiprows = 11)

Additional challenges in this dataset:  
1) Index needs to be reset and forward filled to remove the NAN values from the index  
2) When extracting the data from ABS, although I filtered on Sydney, it still gave me a list of all suburbs. The difference here is that for suburbs that are not in the Greater Sydney area, the population is 0. This should be easier to remove

In [8]:
#Reset Index
suburb_cbop.reset_index(inplace = True)
#Forward fill the country name and copy only the country column. We need the NaNs for cleaning purposes
suburb_cbop['index'] = suburb_cbop.ffill()['index']
suburb_cbop

,index,BPLP - 4 Digit Level,SSC (UR),Unnamed: 2
0,England,Aarons Pass,0.0,NaN
1,England,Abbotsbury,40.0,NaN
2,England,Abbotsford (NSW),181.0,NaN
3,England,Abercrombie,0.0,NaN
4,England,Abercrombie River,0.0,NaN
...,...,...,...,...
31681,Total,Migratory - Offshore - Shipping (NSW),0.0,NaN
31682,"Data Source: Census of Population and Housing,...",NaN,NaN,NaN
31683,INFO,Cells in this table have been randomly adjuste...,NaN,NaN
31684,"Copyright Commonwealth of Australia, 2018, see...",NaN,NaN,NaN


In [9]:
#Set index back to country
suburb_cbop.set_index('index', inplace = True)
suburb_cbop.index.rename('CBOP', inplace = True)

In [10]:
#Removing rows with 2 NaN values
suburb_cbop.dropna(axis = 1, thresh = 2, inplace = True)
suburb_cbop.dropna(axis = 0, thresh = 2, inplace = True)
col_names = ['Suburb', 'Population']
suburb_cbop.columns = col_names
suburb_cbop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31682 entries, England to Total
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Suburb      31682 non-null  object 
 1   Population  31682 non-null  float64
dtypes: float64(1), object(1)
memory usage: 742.5+ KB


Now that we have removed the NaN values, lets look at the numbers and clean those up as well. But it is not as simple as it seems. There could be suburbs in Sydney which do not have any person from one of the top 6 nationalities. In that case, I want to preserve the 0 associated with that particular nationality for my analysis. So before I start cleaning up the numbers, lets first clean up the total suburb population dataframe

In [11]:
suburb_pop.dropna(axis = 1, thresh = 2, inplace = True)
suburb_pop.dropna(axis = 0, thresh = 1, inplace = True)
#Drop info column
suburb_pop.drop('INFO', inplace = True)

suburb_pop.columns = ['Population']
#Convert population to integer
suburb_pop['Population'] = suburb_pop['Population'].apply(pd.to_numeric)
suburb_pop.rename_axis('Suburb', inplace = True)
print(suburb_pop.info())
print("Sydney Population: ", suburb_pop.Population.sum())


<class 'pandas.core.frame.DataFrame'>
Index: 4526 entries, Aarons Pass to Migratory - Offshore - Shipping (NSW)
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Population  4526 non-null   int64
dtypes: int64(1)
memory usage: 70.7+ KB
None
Sydney Population:  4823982


The suburb_pop table now has the population of every suburb in Greater Sydney. For the remaining suburbs the population would be zero. So if we drop the 0 populations we should be left with Sydney suburbs only

In [12]:
#Get index names of suburbs with zero populations
zero_suburbs = suburb_pop[suburb_pop.Population == 0].index
#Drop these index names
suburb_pop.drop(zero_suburbs, inplace=True)
suburb_pop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 893 entries, Abbotsbury to Zetland
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Population  893 non-null    int64
dtypes: int64(1)
memory usage: 14.0+ KB


Dropping the 0 population suburbs leaves us with 893 suburbs. I can also use the same index names to update the top nationalities data frame (*suburb_cbop*). For that I would first need to set the suburb as the index and then drop

In [13]:
suburb_cbop.reset_index(inplace = True)
suburb_cbop.set_index('Suburb', inplace = True)
suburb_cbop.loc[()]

,CBOP,Population
Suburb,,
Aarons Pass,England,0.0
Abbotsbury,England,40.0
Abbotsford (NSW),England,181.0
Abercrombie,England,0.0
Abercrombie River,England,0.0
...,...,...
Yuraygir,Total,0.0
Zara,Total,0.0
Zetland,Total,6618.0


In [14]:
suburb_cbop.drop(zero_suburbs, inplace = True)
suburb_cbop.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6251 entries, Abbotsbury to Zetland
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CBOP        6251 non-null   object 
 1   Population  6251 non-null   float64
dtypes: float64(1), object(1)
memory usage: 146.5+ KB


In [15]:
suburb_cbop.reset_index(inplace = True)

suburb_cbop.set_index(['CBOP','Suburb'], inplace = True)
suburb_cbop

Population
CBOP    Suburb                                        
England Abbotsbury                                40.0
        Abbotsford (NSW)                         181.0
        Acacia Gardens                            63.0
        Agnes Banks                               23.0
        Airds                                     46.0
...                                                ...
Total   Yellow Rock (Blue Mountains - NSW)       941.0
        Yennora                                  868.0
        Yerranderie                                3.0
        Yowie Bay                               2725.0
        Zetland                                 6618.0

[6251 rows x 1 columns]

In [16]:
print(suburb_cbop.loc['Total', 'Zetland'])
print(suburb_pop.loc['Zetland'])
suburb_cbop.loc()

Population    6618.0
Name: (Total, Zetland), dtype: float64
Population    10072
Name: Zetland, dtype: int64


Finally we are now in a position to create an others row for each suburb in Sydney. The others row will contain the number of people who are not from the top 6 countries

In [17]:
#Joining the suburb top 6 with suburb totap population dataframes and extracting the 'Total' index only
others_temp = suburb_cbop.join(suburb_pop, on='Suburb', rsuffix = "r").loc[('Total', slice(None))]
#Calculating the others population value
others_temp.Population = others_temp.Populationr - others_temp.Population
others_temp.drop(['Populationr'], axis=1, inplace = True)
#Creating a final dataframe with others population value in Population_r column
suburb_cbop_final = suburb_cbop.join(others_temp, on='Suburb', rsuffix = '_r')
#Setting the Total index value to Nan
suburb_cbop_final.loc[('Total', slice(None)),'Population'] = np.nan
#renaming index value Total to Others
suburb_cbop_final.rename(index={'Total':'Others'}, inplace = True)
#Setting the right population value in the Others index
suburb_cbop_final['Population'] = np.where(suburb_cbop_final['Population'].isnull(),
                                          suburb_cbop_final['Population_r'],
                                          suburb_cbop_final['Population'])
suburb_cbop_final.drop(['Population_r'], axis=1, inplace = True)
suburb_cbop_final

Population
CBOP    Suburb                                        
England Abbotsbury                                40.0
        Abbotsford (NSW)                         181.0
        Acacia Gardens                            63.0
        Agnes Banks                               23.0
        Airds                                     46.0
...                                                ...
Others  Yellow Rock (Blue Mountains - NSW)        94.0
        Yennora                                  751.0
        Yerranderie                                3.0
        Yowie Bay                                349.0
        Zetland                                 3454.0

[6251 rows x 1 columns]

That took a lot of steps and although I have setup what I wanted to, I refuse to believe that I cant find a more efficient way to do this calculation. Lets try this again

In [18]:
#resetting the index in the suburb cbop file
suburb_cbop.reset_index(inplace = True)
suburb_cbop.set_index('Suburb', inplace=True)
suburb_cbop_final = suburb_cbop.join(suburb_pop, on='Suburb', rsuffix = "r")
suburb_cbop_final['Population'] = np.where(suburb_cbop_final['CBOP'] == 'Total',
                                    suburb_cbop_final['Populationr'] - suburb_cbop_final['Population'],
                                    suburb_cbop_final['Population'])
suburb_cbop_final.drop(['Populationr'], axis=1, inplace = True)
suburb_cbop_final

,CBOP,Population
Suburb,,
Abbotsbury,England,40.0
Abbotsford (NSW),England,181.0
Acacia Gardens,England,63.0
Agnes Banks,England,23.0
Airds,England,46.0
...,...,...
Yellow Rock (Blue Mountains - NSW),Total,94.0
Yennora,Total,751.0
Yerranderie,Total,3.0


In [19]:
#As a final step, renaming Total to Others
suburb_cbop_final.reset_index(inplace=True)
suburb_cbop_final.set_index('CBOP', inplace = True)
suburb_cbop_final.rename({'Total': 'Others'}, inplace = True)

That was simpler. Not I will save my clean files for part two.

In [20]:
suburb_cbop_final

,Suburb,Population
CBOP,,
England,Abbotsbury,40.0
England,Abbotsford (NSW),181.0
England,Acacia Gardens,63.0
England,Agnes Banks,23.0
England,Airds,46.0
...,...,...
Others,Yellow Rock (Blue Mountains - NSW),94.0
Others,Yennora,751.0
Others,Yerranderie,3.0


In [21]:
sydney_cbop.to_csv('Sydney_CBOP_Clean.csv')
suburb_cbop_final.to_csv('Sydney_Suburb_CBOP_Clean.csv')